# Predicting Employee Attrition

Installing and Importing Libaries 

In [35]:
#!pip install --force-reinstall setuptools
#!pip install ydata-profiling --no-cache-dir
#!pip install pandas
#!pip install ydata-profiling
#!pip install ipywidgets

In [36]:
from ydata_profiling import ProfileReport
import pandas as pd

Importing file (csv)

In [37]:
#Import HR dataset
HR_Attrition_path = "../dados/HR_Attrition_Dataset.csv"
df = pd.read_csv(HR_Attrition_path, sep=',', header=0)

print(df.head())

   Age Attrition     BusinessTravel  DailyRate              Department  \
0   41       Yes      Travel_Rarely       1102                   Sales   
1   49        No  Travel_Frequently        279  Research & Development   
2   37       Yes      Travel_Rarely       1373  Research & Development   
3   33        No  Travel_Frequently       1392  Research & Development   
4   27        No      Travel_Rarely        591  Research & Development   

   DistanceFromHome  Education EducationField  EmployeeCount  EmployeeNumber  \
0                 1          2  Life Sciences              1               1   
1                 8          1  Life Sciences              1               2   
2                 2          2          Other              1               4   
3                 3          4  Life Sciences              1               5   
4                 2          1        Medical              1               7   

   ...  RelationshipSatisfaction StandardHours  StockOptionLevel  \
0  ...

## 1. Data Preprocessing

### 1.1. EDA Report with Pandas Profiling (ydata-profiling)

In [38]:
profile = ProfileReport(
    df,
    title="HR Attrition Dataset | EDA Report",
    minimal=True,  
    explorative=True,
    progress_bar=False 
)
profile.to_notebook_iframe()



100%|██████████| 35/35 [00:00<00:00, 400.70it/s]


### 1.2. Checking Data Consistency

In [39]:
def check_consistency(df, col1, col2, condition, description):
    """
    Validates condition between two columns.
    """
    inconsistent = df[~condition(df)]
    count = len(inconsistent)

    print(f"\n{description}")
    print(f"Inconsistent: {count}")

    if count > 0:
        print("Sample:")
        display(inconsistent.head())
    else:
        print("All consistent")

# Validation rules
rules = [
    ("TotalWorkingYears", "YearsAtCompany", 
     lambda d: d["TotalWorkingYears"] >= d["YearsAtCompany"]),
    
    ("YearsAtCompany", "YearsInCurrentRole",
     lambda d: d["YearsAtCompany"] >= d["YearsInCurrentRole"]),
    
    ("YearsAtCompany", "YearsWithCurrManager",
     lambda d: d["YearsAtCompany"] >= d["YearsWithCurrManager"]),
    
    ("YearsAtCompany", "YearsSinceLastPromotion",
     lambda d: d["YearsAtCompany"] >= d["YearsSinceLastPromotion"])
]

# Apply rules
for col1, col2, condition in rules:
    if col2 in df.columns:
        check_consistency(df, col1, col2, condition, f"{col1} >= {col2}")

# Summary by category
print("\nSUMMARY BY NUMCOMPANIESWORKED")

for num in sorted(df['NumCompaniesWorked'].unique()):
    subset = df[df['NumCompaniesWorked'] == num]
    total = len(subset)
    
    if num == 0:
        consistent = subset[subset['TotalWorkingYears'] == subset['YearsAtCompany']]
        pct = (len(consistent) / total) * 100 if total > 0 else 0
        diff = (subset['TotalWorkingYears'] - subset['YearsAtCompany']).abs().mean()
        print(f"Companies {num}: {len(consistent)}/{total} consistent ({pct:.1f}%) | Diff: {diff:.1f}")
    else:
        consistent = subset[subset['TotalWorkingYears'] >= subset['YearsAtCompany']]
        pct = (len(consistent) / total) * 100 if total > 0 else 0
        print(f"Companies {num}: {len(consistent)}/{total} consistent ({pct:.1f}%)")


TotalWorkingYears >= YearsAtCompany
Inconsistent: 0
All consistent

YearsAtCompany >= YearsInCurrentRole
Inconsistent: 0
All consistent

YearsAtCompany >= YearsWithCurrManager
Inconsistent: 0
All consistent

YearsAtCompany >= YearsSinceLastPromotion
Inconsistent: 0
All consistent

SUMMARY BY NUMCOMPANIESWORKED
Companies 0: 0/197 consistent (0.0%) | Diff: 1.0
Companies 1: 521/521 consistent (100.0%)
Companies 2: 146/146 consistent (100.0%)
Companies 3: 159/159 consistent (100.0%)
Companies 4: 139/139 consistent (100.0%)
Companies 5: 63/63 consistent (100.0%)
Companies 6: 70/70 consistent (100.0%)
Companies 7: 74/74 consistent (100.0%)
Companies 8: 49/49 consistent (100.0%)
Companies 9: 52/52 consistent (100.0%)


### 1.3. Removal of irrelevant features

In [40]:
# Remove irrelevant columns
cols_to_drop = ['EmployeeCount', 'Over18', 'StandardHours']
existing_cols_to_drop = [col for col in cols_to_drop if col in df.columns]
df.drop(columns=existing_cols_to_drop, inplace=True, errors='ignore')
print(f"Removed columns: {existing_cols_to_drop}")

# Remove duplicates
duplicates = df.duplicated().sum()
print(f"Duplicate rows: {duplicates}")

if duplicates > 0:
    df = df.drop_duplicates()
    print(f"Duplicates removed. New shape: {df.shape}")

print(df)

Removed columns: ['EmployeeCount', 'Over18', 'StandardHours']
Duplicate rows: 0
      Age Attrition     BusinessTravel  DailyRate              Department  \
0      41       Yes      Travel_Rarely       1102                   Sales   
1      49        No  Travel_Frequently        279  Research & Development   
2      37       Yes      Travel_Rarely       1373  Research & Development   
3      33        No  Travel_Frequently       1392  Research & Development   
4      27        No      Travel_Rarely        591  Research & Development   
...   ...       ...                ...        ...                     ...   
1465   36        No  Travel_Frequently        884  Research & Development   
1466   39        No      Travel_Rarely        613  Research & Development   
1467   27        No      Travel_Rarely        155  Research & Development   
1468   49        No  Travel_Frequently       1023                   Sales   
1469   34        No      Travel_Rarely        628  Research & Development

### 1.3. Indexing

In [41]:
# Definying 'EmployeeNumber' as the index

employee_number_only= df["EmployeeNumber"]
df.set_index("EmployeeNumber", inplace=True)

### 1.4. Transformation of categorical features

In [42]:
df_nominal_categorical= df[[
    "Attrition", "Department","EducationField", "Gender","JobRole", "MaritalStatus", "OverTime"]
].copy()

In [43]:
df_ordinal_categorical= df[[
    "Education", "JobInvolvement", "JobLevel", 
    "JobSatisfaction", "PerformanceRating",
    "RelationshipSatisfaction", "StockOptionLevel", "WorkLifeBalance", "EnvironmentSatisfaction", "BusinessTravel"]
].copy()

In [44]:
#Definying categorical and quantitative features
df_categorical = df[[
    "Attrition", "BusinessTravel", "Department", "Education", "EducationField", "Gender", "JobInvolvement", "JobLevel", "JobRole",
    "JobSatisfaction", "MaritalStatus", "OverTime", "PerformanceRating",
    "RelationshipSatisfaction", "StockOptionLevel", "WorkLifeBalance", "EnvironmentSatisfaction"
]
].copy()

df_categorical.info()

<class 'pandas.core.frame.DataFrame'>
Index: 1470 entries, 1 to 2068
Data columns (total 17 columns):
 #   Column                    Non-Null Count  Dtype 
---  ------                    --------------  ----- 
 0   Attrition                 1470 non-null   object
 1   BusinessTravel            1470 non-null   object
 2   Department                1470 non-null   object
 3   Education                 1470 non-null   int64 
 4   EducationField            1470 non-null   object
 5   Gender                    1470 non-null   object
 6   JobInvolvement            1470 non-null   int64 
 7   JobLevel                  1470 non-null   int64 
 8   JobRole                   1470 non-null   object
 9   JobSatisfaction           1470 non-null   int64 
 10  MaritalStatus             1470 non-null   object
 11  OverTime                  1470 non-null   object
 12  PerformanceRating         1470 non-null   int64 
 13  RelationshipSatisfaction  1470 non-null   int64 
 14  StockOptionLevel          147

In [45]:
education_mapping = {1: "Below College", 2: "College", 3: "Bachelor", 4: "Master", 5: "Doctor"}
job_involvement_mapping = {1: "Low", 2: "Medium", 3: "High", 4: "Very High"}
job_level_mapping = {1: "HC Rep", 2: "HR", 3: "Lab Technician", 4: "Manager", 5: "Managing Director", 6: "Research Director"}
job_satisfaction_mapping = {1: "Low", 2: "Medium", 3: "High", 4: "Very High"}
performance_rating_mapping = {1: "Low", 2: "Good", 3: "Excellent", 4: "Outstanding"}
relationship_satisfaction_mapping = {1: "Low", 2: "Medium", 3: "High", 4: "Very High"}
stock_option_mapping = {0: "None", 1: "Low", 2: "Medium", 3: "High"}
work_life_balance_mapping = {1: "Bad", 2: "Good", 3: "Better", 4: "Best"}
environment_satisfaction_mapping = {1: "Low", 2: "Medium", 3: "High", 4: "Very High"}
business_travel_mapping = {'Non-Travel': 0, 'Travel_Frequently': 2, 'Travel_Rarely': 1}

# Aplicar os mapeamentos no DataFrame
df_categorical["Education"] = df_categorical["Education"].map(education_mapping)
df_categorical["JobInvolvement"] = df_categorical["JobInvolvement"].map(job_involvement_mapping)
df_categorical["JobLevel"] = df_categorical["JobLevel"].map(job_level_mapping)
df_categorical["JobSatisfaction"] = df_categorical["JobSatisfaction"].map(job_satisfaction_mapping)
df_categorical["PerformanceRating"] = df_categorical["PerformanceRating"].map(performance_rating_mapping)
df_categorical["RelationshipSatisfaction"] = df_categorical["RelationshipSatisfaction"].map(relationship_satisfaction_mapping)
df_categorical["StockOptionLevel"] = df_categorical["StockOptionLevel"].map(stock_option_mapping)
df_categorical["WorkLifeBalance"] = df_categorical["WorkLifeBalance"].map(work_life_balance_mapping)
df_categorical["EnvironmentSatisfaction"] = df_categorical["EnvironmentSatisfaction"].map(environment_satisfaction_mapping)

#Check df
df_categorical.info()

<class 'pandas.core.frame.DataFrame'>
Index: 1470 entries, 1 to 2068
Data columns (total 17 columns):
 #   Column                    Non-Null Count  Dtype 
---  ------                    --------------  ----- 
 0   Attrition                 1470 non-null   object
 1   BusinessTravel            1470 non-null   object
 2   Department                1470 non-null   object
 3   Education                 1470 non-null   object
 4   EducationField            1470 non-null   object
 5   Gender                    1470 non-null   object
 6   JobInvolvement            1470 non-null   object
 7   JobLevel                  1470 non-null   object
 8   JobRole                   1470 non-null   object
 9   JobSatisfaction           1470 non-null   object
 10  MaritalStatus             1470 non-null   object
 11  OverTime                  1470 non-null   object
 12  PerformanceRating         1470 non-null   object
 13  RelationshipSatisfaction  1470 non-null   object
 14  StockOptionLevel          147

In [46]:
df_categorical['OverTime'] = df_categorical['OverTime'].map({"No": 0, "Yes": 1})
df_categorical["Attrition"] = df_categorical["Attrition"].map({"No": 0, "Yes": 1})

In [47]:
df_quantitative = df[[
 "Age", "DailyRate", "DistanceFromHome", "HourlyRate", "MonthlyIncome",
    "MonthlyRate", "NumCompaniesWorked", "PercentSalaryHike", "TotalWorkingYears",
    "TrainingTimesLastYear", "YearsAtCompany", "YearsInCurrentRole",
    "YearsSinceLastPromotion", "YearsWithCurrManager"
]].copy()

df_quantitative.info()

<class 'pandas.core.frame.DataFrame'>
Index: 1470 entries, 1 to 2068
Data columns (total 14 columns):
 #   Column                   Non-Null Count  Dtype
---  ------                   --------------  -----
 0   Age                      1470 non-null   int64
 1   DailyRate                1470 non-null   int64
 2   DistanceFromHome         1470 non-null   int64
 3   HourlyRate               1470 non-null   int64
 4   MonthlyIncome            1470 non-null   int64
 5   MonthlyRate              1470 non-null   int64
 6   NumCompaniesWorked       1470 non-null   int64
 7   PercentSalaryHike        1470 non-null   int64
 8   TotalWorkingYears        1470 non-null   int64
 9   TrainingTimesLastYear    1470 non-null   int64
 10  YearsAtCompany           1470 non-null   int64
 11  YearsInCurrentRole       1470 non-null   int64
 12  YearsSinceLastPromotion  1470 non-null   int64
 13  YearsWithCurrManager     1470 non-null   int64
dtypes: int64(14)
memory usage: 172.3 KB


In [48]:
df_nominal_categorical = df[[
'Attrition', 'Department', 'EducationField', 'Gender', 'JobRole', 'MaritalStatus', 'OverTime'
]].copy()

In [49]:
df_ordinal_categorical= df[[
    "Education", "JobInvolvement", "JobLevel", 
    "JobSatisfaction", "PerformanceRating",
    "RelationshipSatisfaction", "StockOptionLevel", "WorkLifeBalance", "EnvironmentSatisfaction", "BusinessTravel"]
].copy()

In [50]:
# Set EmployeeNumber as the index for categorical and quantitative df
df_categorical.set_index(employee_number_only, inplace=True)
df_quantitative.set_index(employee_number_only, inplace=True)

## 2. 